# 03 - Assemble the RAG Pipeline

Notebook steps:

   - Load the vector database retriever and test it
   - Initialize the reader with a prompt template and a LLM
   - Some retrievers optimizations: hybrid search and reranking
   - Create a RAG pipeline that combines the retriever and the reader


In [3]:
import os
import time
import asyncio
import logging
import re
from typing import List, Tuple
from collections import defaultdict

import pandas as pd
import language_tool_python

import lmstudio as lms
from nltk.tokenize import sent_tokenize
from langchain_openai import ChatOpenAI
from langchain_core.documents import Document

from IPython.display import display, Markdown
from jinja2 import Template

from lib.vector_store import VectorDB
from lib.io_utils import get_absolute_path

## Retriever

In the previous notebook, we created the vector database that works in the RAG as a boosted search engine that brings up the most relevant documents in relation to a user query.

We call it the retriever in the RAG.

We init the retriever, to understand how it works, we can look at the notebook `notebooks/scripts/02-build_vectordb.ipynb` which allows us to create a vectordb from a dataframe.


In [6]:
# init variables for retriever just for example (check available in config.yml for more details)
backend = "lancedb"
embedding_model_name = "Lajavaness/sentence-camembert-large"
db_store_path = get_absolute_path("data/retrievers/vectordb/camembert-large_lancedb")

db = VectorDB(
    backend=backend,
    embedding_model=embedding_model_name,
    path=db_store_path
)

📦 LanceDB intialization on: /Users/lucaterre/Documents/pro/Travail_courant/DEV/AI-ENC-Projects/on-github/encpos-qa-rag/data/retrievers/vectordb/camembert-large_lancedb
ℹ️ LanceDB table founded.


Let's try the retriever with a query.

In [8]:
def group_by_thesis(results_with_score: list[tuple[Document, float]]) -> dict[str, list[tuple[Document, float]]]:
    """Group the results by thesis file_id.

    Args:
        results_with_score (list[tuple[Document, float]]): List of tuples containing Document and score.

    Returns:
        dict[str, list[tuple[Document, float]]]: Dictionary with file_id as keys and list of (Document, score) tuples as values.
    """
    grouped = defaultdict(list)
    for doc, score in results_with_score:
        file_id = doc.metadata.get("file_id", "unknown")
        grouped[file_id].append((doc, score))
    return grouped

def readeable_output_display(grouped_results: dict[str, list[tuple[Document, float]]]) -> None:
    """ Display the grouped results in a readable format.

    Args:
        grouped_results (dict[str, list[tuple[Document, float]]]): Dictionary with file_id as keys and list of (Document, score) tuples as values.
    Returns:
        None: This function prints the results directly.
    """
    for file_id, docs in grouped_results.items():
        # Sort documents by score
        docs = sorted(docs, key=lambda tup: tup[1], reverse=True)

        titre = docs[0][0].metadata.get("position_name", "Sans titre")
        auteur = docs[0][0].metadata.get("author", "Inconnu")
        date = docs[0][0].metadata.get("year", "Inconnu")

        print(f"\n📄 {auteur}, {titre}, promotion {date}")
        print(f"🧩 Chunks founded : {len(docs)}\n")

        for i, (doc, score) in enumerate(docs):
            extrait = doc.metadata.get("raw_chunk", doc.page_content).strip().replace("\n", " ")
            section = doc.metadata.get("section", "Inconnu")
            print(f"  ▪️ Extract {i + 1} from section '{section}' (score={score:.4f}) : […]{extrait}[…]")
        print("-" * 80)

def mini_retriever_playground(k: int=10)-> None:
    """Mini retriever playground to test the retriever with user input.

    Args:
        k (int): Number of results to return. Defaults to 10.

    Returns:
        None: This function runs an interactive loop to query the retriever.
    """
    query = input("Query: ")

    results = db.query(query, k=k)
    grouped_results = group_by_thesis(results)
    print(f"User query: {query}\n")
    readeable_output_display(grouped_results)
    return query, results


In [9]:
%%time
query, results = mini_retriever_playground(k=10)

User query: Quels sont les thèmes iconographiques au Moyen-Âge ?


📄 Emmanuelle Giry, Léon Bloy et le Moyen Âge l’imaginaire catholique renouvelé ?, promotion 2011
🧩 Chunks founded : 1

  ▪️ Extract 1 from section 'Annexes' (score=0.5563) : […]Iconographie. — Index des personnages médiévaux cités dans l’œuvre de Bloy. — Tableau synoptique des principaux historiens lus par Bloy. — Index général.[…]
--------------------------------------------------------------------------------

📄 Jacques Yvon, L’illustration des romans arthuriens du xiiie au xve siècle, promotion 1948
🧩 Chunks founded : 1

  ▪️ Extract 1 from section 'Chapitre III L’iconographie.' (score=0.5548) : […]L’iconographie profane a été peu étudiée. Elle a fait des emprunts à l’iconographie religieuse qui sont visibles dans l’illustration des romans de chevalerie. Certains thèmes religieux ont été traités par les enlumineurs de romans, comme l’Annonciation, la Crucifixion. Les illustrations peuvent apporter des renseignements 

## Reader

The reader is the RAG component that takes the results from the retriever and transforms them (interpolation) into a pre-structured prompt, which is then passed to an LLM that generates a structured or unstructured response.

The parameters to consider for the reader are:
- The LLM model that will be used to generate the response;
- The prompt that will be used to generate the response;

To initialize the Reader, we use LangChain's `ChatOpenAI` interface, which allows us to use OpenAI's LLM models (or models compatible with the OpenAI API specification, such as Mistral, Llama, etc.).
The LLM is served by LM Studio, which is a server compatible with the OpenAI API and allows you to easily use local LLM models (there are other solutions such as Ollama or Vllm, for example).

To initialize the LLM, you must specify (see the documentation https://python.langchain.com/docs/integrations/chat/openai/):
- The maximum number of tokens to be generated by the LLM (`max_tokens`);
- The OpenAI API URL (which is actually the LM Studio server URL) (`openai_api_base`);
- The response will be streamed (`streaming=True`) to display the response as it is generated.
- max_retries: the number of generation attempts in case of error (for example, if the LLM does not respond within a reasonable time) (`max_retries`).
- timeout: the maximum waiting time for a response from the LLM (`timeout`).

Inference parameters (set directly in lmstudio):

- The temperature of the LLM (for the creativity of the response). A temperature of 0.0 will make the LLM very deterministic, while a temperature of 1.0 will make it more creative and random (`temperature`);
- The top_p parameter (which is not used here, but can be useful to control the diversity of the generated response) (`top_p`);


In [22]:
llm_name = "mistral-nemo-instruct-2407"

llm_only = lms.list_loaded_models("llm")
if len(llm_only) > 0:
    print("The following models were loaded:")
    for model in llm_only:
        print(f"- {model}")
    llm = ChatOpenAI(
    model_name="mistral-nemo-instruct-2407",
    openai_api_base="http://localhost:1234/v1",
    openai_api_key="lm-studio",
    streaming=True,
    max_tokens=4096,
)
else:
    print("No LLM models loaded in LM Studio. Try to load a new instance.")
    client = lms.get_default_client()
    client.llm.unload(llm_name)
    model = client.llm.load_new_instance(llm_name)
    llm = ChatOpenAI(
    model_name="mistral-nemo-instruct-2407",
    openai_api_base="http://localhost:1234/v1",
    openai_api_key="lm-studio",
    streaming=True,
    max_tokens=4096,
)


The following models were loaded:
- LLM(identifier='mistral-nemo-instruct-2407')


### Reader - the prompt

In [10]:
def get_token_count(text: str) -> int:
    """Get the number of tokens in a text using the LLM tokenizer.
    Args:
        text (str): The text to tokenize.
    Returns:
        int: The number of tokens in the text.
    """
    model = lms.llm(
    )
    return len(model.tokenize(text))

def truncate_by_sentence(text: str, max_tokens: int) -> str:
    """Truncate a text by sentences to fit within a maximum token limit.

    Args:
        text (str): The text to truncate.
        max_tokens (int): The maximum number of tokens allowed.

    Returns:
        str: The truncated text, ending with an ellipsis if truncated.
    """
    sentences = sent_tokenize(text)
    result = []
    for sent in sentences:
        result.append(sent)
        joined = " ".join(result)
        if get_token_count(joined) > max_tokens:
            result.pop()
            break
    return " ".join(result).strip() + " […]"

def build_context_prompt(
    results: List[Tuple[Document, float]],
    question: str,
    template_path: str,
    max_total_tokens: int = 4096,
    output_buffer: int = 512,
) -> str:
    """Build a context prompt from the retrieved documents and a question.

    Args:
        results (List[Tuple[Document, float]]): List of tuples containing Document and score.
        question (str): The question to be answered.
        template_path (str): Path to the Jinja2 template file for the prompt.
        max_total_tokens (int): Maximum total tokens allowed for the prompt.
        output_buffer (int): Buffer space reserved for the output tokens.

    Returns:
        str: The rendered prompt with context and question.
    """
    prompt_template = Template(open(template_path, encoding="utf-8").read())

    grouped = defaultdict(list)
    for doc, score in results:
        grouped[doc.metadata.get("file_id", "inconnu")].append((doc, score))

    # test if all score is under < 0.5:
    if all(score < 0.5 for _, score in results):
        return "no documents found"

    sorted_groups = sorted(grouped.items(), key=lambda item: max(s for _, s in item[1]), reverse=True)

    included_chunks = []
    fallback_chunks = []
    token_budget = max_total_tokens - output_buffer

    header_base = prompt_template.render(context="PLACEHOLDER", question="PLACEHOLDER", annex="PLACEHOLDER")
    header_tokens = get_token_count(header_base.replace("{{context}}", "").replace("{{question}}", ""))

    total_tokens = header_tokens

    for file_id, chunks in sorted_groups:
        chunks.sort(key=lambda x: x[1], reverse=True)
        meta = chunks[0][0].metadata
        header = f"* Position de thèse : {meta.get('author','?')}, {meta.get('position_name','?')}, promotion {meta.get('year','?')}\n"
        section_lines = [header]

        for i, (doc, _) in enumerate(chunks):
            section = doc.metadata.get("section", "")
            extrait = doc.metadata.get("raw_chunk", doc.page_content).strip().replace("\n", " ")
            #extrait = truncate_by_sentence(extrait, max_chunk_tokens)

            line = f"Extrait {i+1}"
            if section:
                line += f" - section « {section} »"
            line += f" : {extrait}"

            chunk_tokens = get_token_count(line)
            if total_tokens + chunk_tokens > token_budget:
                fallback_chunks.append((meta, i+1, section))
                continue

            section_lines.append(line)
            total_tokens += chunk_tokens

        if len(section_lines) > 1:
            included_chunks.append("\n".join(section_lines) + "\n")

    context = "\n".join(included_chunks)

    # Thèses éloignées
    if fallback_chunks:
        annex_by_thesis = defaultdict(list)
        for meta, i, section in fallback_chunks:
            file_id = meta.get("file_id", "inconnu")
            annex_by_thesis[file_id].append((meta, i, section))

        annex_lines = []
        for file_id, chunk_infos in annex_by_thesis.items():
            meta = chunk_infos[0][0]
            title = meta.get("position_name", "?")
            author = meta.get("author", "?")
            promo = meta.get("year", "?")
            header = f"* {author}, {title}, promotion {promo} :"
            annex_lines.append(header)

            for _, i, section in chunk_infos:
                line = f"\t- "
                if section:
                    line += f"section « {section} »"
                annex_lines.append(line)

        annex = "\n".join(annex_lines)
    else:
        annex = None

    return prompt_template.render(context=context, question=question, annex=annex)

In [11]:
final_prompt = build_context_prompt(
    results=results,
    question=query,
    template_path="../prompt_templates/v2.jinja",
    max_total_tokens=4096,
    output_buffer=512,
)

print(final_prompt)

**Votre identité et objectif principal:**
Vous êtes un assistant IA spécialisé dans l'exploration et l'analyse du corpus des "Positions des thèses" de l'École nationale des chartes, publiées depuis 1849. Votre rôle est d'aider les utilisateurs à naviguer, comprendre et extraire des informations pertinentes de ces documents historiques. Vous devez baser vos réponses **exclusivement** sur les informations contenues dans les "Positions des thèses" qui vous sont fournies.

Rappelez clairement à l'utilisateur, notamment au début de l'interaction et chaque fois que le contexte le justifie (par exemple, si la question implique une recherche d'exhaustivité ou si l'information trouvée est très concise), que vos réponses sont basées exclusivement sur les 'positions de thèse'. Précisez que celles-ci sont des résumés et que, pour une compréhension approfondie ou complète, la consultation de la thèse originale est recommandée.

**Nature des données :**
1.  Vous avez accès au contenu textuel intégra

### Reader - try it

We pass the prompt to the LLM to generate a response. The response will be streamed, meaning that it will be displayed as it is generated, allowing for a more interactive experience.

We add some post-processing operations on generated content such as:

- Spell check to avoid spelling mistakes in the response due to a small LLM with a strategy to prevent overcorrection by naive named entities masking;
- Avoid LLM abrupt cutoffs on the middle of sentence by splitting the response to the last sentence;
- Add a user disclaimer at the end of the response to remind the user that the response is generated by an LLM and may contain errors or approximations.

In [31]:
os.environ["TOKENIZERS_PARALLELISM"] = "false"
tool = language_tool_python.LanguageTool('fr')

def mask_capitalized_words(text: str) -> Tuple[str, dict]:
    """Mask all words starting with a capital letter in the text.

    Args:
        text (str): The input text to mask.

    Returns:
        Tuple[str, dict]: A tuple containing the masked text and a dictionary of masks.
    """
    masks = {}
    def replacer(match):
        key = f"__MASK{len(masks)}__"
        masks[key] = match.group(0)
        return key

    # Mask all the words that start with a capital letter
    pattern = r'\b[A-ZÉÈÀÂÊÎÔÛÇ][a-zéèêàçîôûäëïöü]+\b'
    masked_text = re.sub(pattern, replacer, text)
    return masked_text, masks

def unmask_text(text: str, masks: dict) -> str:
    """Unmask the text by replacing the masked words with their original values.

    Args:
        text (str): The masked text.
        masks (dict): A dictionary containing the masks.
    Returns:
        str: The unmasked text with original words restored.
    """
    for key, original in masks.items():
        text = text.replace(key, original)
    return text

# Disclamer
CONCLUSION = (
    "Pour rappel, cette réponse est générée automatiquement à partir d'un modèle de langue elle peut contenir des approximations, des surcorrections, des erreurs factuelles "
    "ou des interprétations partielles. Cette réponse ne prétend pas se subsituer à la critique des sources. Dans ce cadre, il est vivement recommandé de vérifier les sources mentionnées "
    "dans cette réponse et de consulter d'autres positions de thèses pour approfondir votre question."
)

def finalize_response(text: str, conclusion: str = CONCLUSION) -> str:
    """Post-processing operations on the generated text to clean it up and prepare it for display.

    Args:
        text (str): The generated text to be cleaned.
        conclusion (str): The conclusion to be appended at the end of the response.

    Returns:
        str: The cleaned text with sentences properly formatted and the conclusion appended.
    """
    lines = text.strip().split('\n')
    final_lines = []

    for line in lines:
        sentences = re.split(r'(?<=[.!?])\s+', line)
        if not sentences:
            continue
        if not re.search(r'[.!?]["”»”]?\s*$', sentences[-1]):
            sentences = sentences[:-1]
        if sentences:
            final_lines.append(" ".join(sentences))

    cleaned_text = "\n".join(final_lines).strip()
    if not cleaned_text:
        return conclusion

    masked_text, masks = mask_capitalized_words(cleaned_text)
    matches = tool.check(masked_text)
    corrected = language_tool_python.utils.correct(masked_text, matches)
    corrected_text = unmask_text(corrected, masks)

    return corrected_text + "\n\n" + conclusion


async def stream_and_print(llm, prompt: str, max_tokens: int=100000, timeout: int=100000) -> str:
    """Launch the generation process and display the response.

    Args:
        llm: The LLM model instance to use for generation.
        prompt (str): The prompt to send to the LLM.
        max_tokens (int): Maximum number of tokens allowed in the response.
        timeout (int): Maximum time to wait for the response.

    Returns:
        str: The final cleaned response from the LLM.
    """
    full_response = ""
    output_display = display(Markdown("🟡 Processing..."), display_id=True)

    async def _run_stream():
        nonlocal full_response
        chunks = []
        async for chunk in llm.astream(prompt):
            text = getattr(chunk, "content", str(chunk))
            full_response += text
            chunks.append(text)
            output_display.update(Markdown("".join(chunks).replace("\n", "\n\n")))
            if len(full_response.split()) > max_tokens:
                output_display.update(Markdown("⛔️ **Truncated response: too many tokens.**"))
                break

    try:
        await asyncio.wait_for(_run_stream(), timeout=timeout)
    except asyncio.TimeoutError:
        logging.warning("⏳ Timeout — relaunch in stream...")
        output_display.update(Markdown("🔁 **timeout...**"))
        full_response = ""
        try:
            await asyncio.wait_for(_run_stream(), timeout=timeout)
        except Exception as e:
            logging.error(f"❌ Failed to stream : {e}")
            output_display.update(Markdown("❌ **No generation possible.**"))
            return "Erreur"
    except Exception as e:
        logging.warning(f"💥 Error during streaming: {e}")
        output_display.update(Markdown("🔁 **Test again...**"))
        full_response = ""
        try:
            await asyncio.wait_for(_run_stream(), timeout=timeout)
        except Exception as e:
            logging.error(f"❌ Failed on second test: {e}")
            output_display.update(Markdown("❌ **No generation possible**"))
            return "Erreur"

    # Nettoyage et finalisation
    cleaned_response = finalize_response(full_response)
    output_display.update(Markdown(cleaned_response.replace("\n", "\n\n")))
    return cleaned_response

# Utilisation
if final_prompt == "no documents found":
    response = "Je n'ai trouvé aucun document pertinent pour répondre à votre question. Veuillez reformuler votre question ou bien consulter les positions de thèses disponibles."
    # stream this
    output_display = display(Markdown(response), display_id=True)
else:
    response = await stream_and_print(llm, final_prompt)


D'après les positions de thèse que j'ai consultées, plusieurs thèmes iconographiques étaient courants au Moyen Âge. Les principale thématique concernent l'histoire religieuse, les événements historiques, les portraits et les scènes de genre.

1. **Religion** : Les positions de thèse mentionnent souvent des sujets liés à la religion, tels que la Crucifixion, l'Annonciation, la Vierge à l'Enfant, les saints, les scènes bibliques et les thèmes religieux dans les romans de chevalerie.

2. **Histoire** : L'histoire est un autre sujet populaire pour l'iconographie au Moyen Âge. Les positions de thèse font référence à des événements historiques, comme des batailles, des couronnements, des mariages royaux, des naissances et des décès de membres de la famille royale.

3. **Portraits** : Les portraits de personnalités, tels que les membres de la royauté, les saints et les héros de l'époque, étaient également courants dans l'iconographie médiévale.

4. **Scènes de genre** : Des scènes de la vie quotidienne, comme le travail des champs, la chasse, le marché, etc., ont aussi été souvent représentées.

Il convient de noter que ces positions de thèse sont des résumés, il est donc possible que les thèses complètes contiennent des informations plus détaillées sur l'iconographie au Moyen Âge.



Pour rappel, cette réponse est générée automatiquement à partir d'un modèle de langue elle peut contenir des approximations, des surcorrections, des erreurs factuelles ou des interprétations partielles. Cette réponse ne prétend pas se subsituer à la critique des sources. Dans ce cadre, il est vivement recommandé de vérifier les sources mentionnées dans cette réponse et de consulter d'autres positions de thèses pour approfondir votre question.

### Reader - optimisations: hybrid search and reranking

There are two possible optimizations:

- Hybrid search: classic information retrieval algorithm (e.g., BM25) combined with a vector retriever (such as FAISS or Qdrant) to improve the relevance of results.
Classic algorithms are often very good at detecting the presence/absence of keywords in documents, while vector algorithms are better at detecting semantic similarity between documents and the question asked.
The two approaches are therefore combined to obtain more relevant results. This is possible via LangChain's `EnsembleRetriever`, which allows several retrievers to be combined into one based on a hybrid search algorithm that performs a weighted combination of the results from each retriever.

- Reranking: this method allows the results obtained by the retriever to be reordered according to their relevance to the question asked, using a language model before the retriever finally returns the results. The idea is that the language model can better understand the context and relevance of the documents in relation to the question asked, and thus reorganize the results to return the most relevant ones first.

Like the VectorDB class, we have created an abstraction `RAGPipeline` that includes the Retriever via `VectorDB`, the logic seen above, and the possibility of using hybrid search and reranking.

In [1]:
from lib.rag_pipeline import RAGPipeline

#### Hybrid search

In [35]:
pipeline = RAGPipeline(
    vectordb_path=get_absolute_path("data/retrievers/vectordb/camembert-base_faiss"),
    template_path=get_absolute_path("prompt_templates/v3.jinja"),
    backend="faiss",
     embedding_model="Lajavaness/sentence-camembert-base",
    hybrid=True,
    bm25_path=get_absolute_path("data/retrievers/bm25/bm25.encpos.tok.512_51.pkl"),
    rerank=False,
    use_streaming=True
)

In [36]:
results = await pipeline.generate(query)
for doc, score in pipeline.relevant_docs:
    print(f"[{score:.2f}] {doc.metadata.get('author', '?')} - {doc.metadata.get('section', '?')}")
    print(doc.page_content[:300], "...\n")

Les extraits fournis permettent d'identifier plusieurs thèmes iconographiques qui ont été couramment utilisés au Moyen Âge. Selon Jacques Yvon dans son étude sur l'illustration des romans arthuriens du XIIIe au XVe siècle, certains thèmes religieux ont été traités par les enlumineurs de ces romans, comme l'Annonciation et la Crucifixion. Ces illustrations peuvent apporter des renseignements précieux sur l'iconographie profane, qui inclut le roi, la guerre, le chevalier et sa vie, etc.

Dominique Grandon, dans ses recherches sur les incunables illustrés, mentionne que les éditions de la Vie de Jésus-Christ appartiennent à un groupe de productions dont le succès marque les débuts de l'imprimerie lyonnaise et genevoise. Ces textes en langue vulgaire, qui connurent une diffusion analogue à celle de notre texte, incluent des romans de chevalerie, des encyclopédies de vulgarisation, etc. L'illustration de ces incunables se compose de séries homogènes de gravures d'un style rudimentaire, mais très réaliste. Les thèmes iconographiques sont des thèmes courants à la fin du Moyen Âge.

En somme, les principaux thèmes iconographiques au Moyen Âge incluent des sujets religieux tels que l'Annonciation et la Crucifixion, ainsi que des thèmes profanes liés aux romans de chevalerie, à la guerre, au roi, au chevalier et à sa vie. Les gravures sur bois utilisées dans les incunables illustrés sont caractéristiques d'un art populaire, vivant et jeune, qui est encore marqué par les contraintes dues à la technique.



Pour rappel, cette réponse est générée automatiquement à partir d'un modèle de langue. Elle peut contenir des approximations, des surcorrections, des erreurs factuelles ou des interprétations partielles. Il est vivement recommandé de vérifier les sources mentionnées et de consulter d'autres positions de thèses pour approfondir votre question.

[1.00] Marie Avril-Lossky - Chapitre V Les icônes à sujets liturgiques aux xvie et xviie siècles. La liturgie des fêtes
Le renouvellement des icônes en Russie aux xvie et xviie siècles : La liturgie de certaines fêtes a suscité d’autres icônes ; ainsi l’Akathiste de la Mère de Dieu a inspiré des illustrations littérales de louanges à la Vierge prises dans le texte de l’office, comme les douze scènes de l’Akathiste qu ...

[1.00] Guy Mayaud - Introduction
L’érudition héraldique au xviie siècle : la question des origines des armoiries : de son enseignement. L’héraldique, science mouvante et évolutive, est donc mise en ordre et structurée par le Grand Siècle. Deux enjeux principaux paraissent dès lors s’imposer dans une étude sur l’érudition héraldique ...

[1.00] Jacques Yvon - Chapitre III L’iconographie.
L’illustration des romans arthuriens du xiiie au xve siècle : L’iconographie profane a été peu étudiée. Elle a fait des emprunts à l’iconographie religieuse qui sont visibles dans l’il

#### Reranking

In [14]:
pipeline = RAGPipeline(
    vectordb_path=get_absolute_path("data/retrievers/vectordb/camembert-base_faiss"),
    template_path=get_absolute_path("prompt_templates/v3.jinja"),
    backend="faiss",
     embedding_model="Lajavaness/sentence-camembert-base",
    hybrid=False,
    rerank=True,
    bm25_path=None,
    use_streaming=True
)

📂 Loading existing FAISS index from /Users/lucaterre/Documents/pro/Travail_courant/DEV/AI-ENC-Projects/on-github/encpos-qa-rag/data/retrievers/vectordb/camembert-base_faiss


In [15]:
results = await pipeline.generate(query)
for doc, score in pipeline.relevant_docs:
    print(f"[{score:.2f}] {doc.metadata.get('author', '?')} - {doc.metadata.get('section', '?')}")
    print(doc.page_content[:300], "...\n")

Les extraits fournis permettent d'aborder la question des thèmes iconographiques au Moyen Âge à travers l'analyse de deux positions de thèse portant sur l'iconographie et l'héraldique.

D'une part, Nathalie Rollet dans son travail sur L’iconographie du Lancelot en prose à la fin du Moyen Âge (v. 1340-v. 1500) met en évidence les thèmes iconographiques liés à la légende du Graal. Elle montre que l'iconographie de la Queste d'EL Saint Graal, qui compte entre trois et quarante-trois miniatures selon les manuscrits, mêle mythe et religion chrétienne. Les chevaliers sont les héros principaux de cette quête, mais ils sont souvent associés aux religieux dans l'illustration. L'auteur souligne également que l'orientation profane de l'iconographie prolonge celle du Lancelot propre, même si le texte de la Queste d'EL Saint Graal insiste moins sur les aventures guerrières des chevaliers que sur l'aspect religieux de leur quête.

D'autre part, Michel Pastoureau dans son travail sur Le bestiaire héraldique au Moyen Âge aborde les thèmes iconographiques liés à l'héraldique. Il montre que le dessin héraldique repose sur une simplification des formes de l'animal et une exagération de toutes les parties pouvant servir à l'identifier. Les règles du dessin héraldique sont rigoureuses, avec des lois de symétrie et de plénitude. On peut distinguer plusieurs styles nationaux dans le dessin héraldique des animaux, tels que le style anglais, le style français, le style germanique et le style bourguignon-flamand.

En synthèse, les thèmes iconographiques au Moyen Âge sont variés et peuvent être liés à la littérature, comme dans le cas de l'iconographie du Lancelot en prose, ou à l'héraldique, comme dans le travail de Michel Pastoureau sur le bestiaire héraldique. Les extraits fournis montrent que ces thèmes iconographiques sont souvent associés à des règles et des styles spécifiques, qui peuvent varier selon les régions et les périodes.



Pour rappel, cette réponse est générée automatiquement à partir d'un modèle de langue. Elle peut contenir des approximations, des surcorrections, des erreurs factuelles ou des interprétations partielles. Il est vivement recommandé de vérifier les sources mentionnées et de consulter d'autres positions de thèses pour approfondir votre question.

[1.00] Nathalie Rollet - Chapitre VI L’iconographie de la Queste del saint Graal
L’iconographie du Lancelot en prose à la fin du Moyen Âge (v. 1340-v. 1500) : Sept manuscrits contiennent le texte de la Queste del Saint Graal. avec un cycle d’images comptant de trois à quarante-trois miniatures et un total de cent quarante images. L’amplification progressive du cycle d’images est ...

[0.96] Anne-Sophie Durozoy - Planches
Étude de l’iconographie et de la théologie de Jonas en Occident (xiie-xve siècle). Jonas au Moyen Âge : Pour chaque partie iconographique, un ensemble d’images illustrant le texte a été rassemblé. ...

[0.92] Mathieu Deldicque - Annexes
Entre Moyen Âge et Renaissance ? La commande artistique de l’amiral Louis Malet de Graville (v. 1440-1516) : Dossier iconographique : planches réparties par édifices et type d’objets. — Généalogies. — Cartes. — Index des noms de personnes et de lieux. ...

[0.91] Nathalie Rollet - Chapitre III Quelques remarques sur le style et la techn